In [1]:
# Import Necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from math import sqrt
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')

In [2]:
# check to see if GPU is used
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6748280101620300124
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7240480511309572560
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5686755328
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2962205596063610444
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17481761659222758138
physical_device_desc: "device: XLA_GPU device"
]


# Model Metrics 

In [3]:
def plot_returns(name, actual, predictions, lookback):
    """Function to plot returns using plotly
    
    Parameters
    ----------
    name: str
        Name to be used for the graph title 
    actual: array
        The test/target labels for 2018
    predictions: array
        The predictions that were made for 2018
    lookback: int
        The lookback used to create LSTM model
    """
    
    predictions = predictions.reshape(len(predictions))
    actual = actual[lookback:]

    assert len(predictions) == len(actual), "Predictions and actual not same length"
    
    fig = go.Figure()
    fig.add_trace(go.Line(y = predictions,
                         name = '{} predictions 2018'.format(name),
                         mode = 'lines'))
    fig.add_trace(go.Line(y = actual,
                         name = 'Actual Returns 2018',
                         mode = 'lines'))
    fig.update_layout(title = '{} model returns vs Actual Returns 2018'.format(name))
    fig.show()

In [4]:
def calc_mae(actual, predictions, lookback):
    """Function to calculate MAE
    
    Parametes
    ---------
    actual: array
        The predictions that were made for 2018
    predictions: array
        The actual/target labels for 2018
        
    Return
    ------
    mae: float
        The calculated MAE
    """
    predictions = predictions.reshape(len(predictions))
    actual = actual[lookback:]

    assert len(predictions) == len(actual), "Predictions and actual not same length"
    
    mae = mean_absolute_error(actual, predictions)
    return mae

In [5]:
class Stacked_LSTM2():
    """A class for a stacked LSTM model
    """
    
    def __init__(self, epochs = 1, lookback = 5, 
                 layer_size = 64, layer2_size = 64):
        self.epochs = epochs
        self.lookback = lookback
        self.layer_size = layer_size
        self.layer2_size = layer2_size
        self.train_gen, self.val_gen, self.test_gen = self.preprocess_data()
        self.model = self.build_compile()

        
    def preprocess_data(self):
        """A function to load data
        """
        
        # Load and preprocess data
        data_path = "../data/ucsbdata.csv"
        raw_data = pd.read_csv(data_path)
        # Select subset of data
        market_crash = '2008-08-30'
        data = raw_data.loc[raw_data.Index > market_crash]
        # Convert Int Values to float 
        non_float_columns = ['URR', 'FOMC', 'PPIR', 'HULBERTSENTIMENT', 'TOM']
        for column in non_float_columns:
            data[column] = data[column].astype(float)
        # convert string Index to datatime
        non_datetime_column = 'Index'
        datetime_values = pd.to_datetime(data[non_datetime_column]).values
        data[non_datetime_column] = datetime_values
        # Replace current index with data index 
        data.set_index(non_datetime_column, inplace = True)
        
        # Create test data frame for the year 2018
        test_year = '2018'
        lookback_year = '2017'
        lookback_2017 = data[lookback_year].iloc[-self.lookback:-1].index.min()
        test_df = data.loc[lookback_2017:test_year]
        
        # Create training data frame for 2008 - 2017
        train_df = data.loc[:lookback_year]
        train_df = train_df.iloc[:-self.lookback]
        
        # Create Test Dataset and test labels
        test_label = 'R'
        test_labels = test_df[test_label].values
        test_data = test_df.iloc[:, 1:].values
        
        # Create Training and validation set: Split with 20% Validation set
        train_labels = train_df[test_label].values
        train_features = train_df.iloc[:, 1:].values
        validation_percentage = 0.2
        train_data, val_data, train_labels, val_labels = train_test_split(
            train_features,
            train_labels,
            test_size = validation_percentage,
            random_state = 0,
            shuffle = False)
        
        # Normalize the data for each feature
        sc = StandardScaler()
        train_data = sc.fit_transform(train_data)
        val_data = sc.transform(val_data)
        test_data = sc.transform(test_data)

        # Data Generator for LSTM model
        sampling_rate = 1
        train_batch_size = 14
        val_batch_size = 14
        test_batch_size = 14
        train_generator = TimeseriesGenerator(train_data, train_labels,
                                             length = self.lookback,
                                             batch_size = train_batch_size,
                                             sampling_rate = sampling_rate,
                                             shuffle = False)
        validation_generator = TimeseriesGenerator(val_data, val_labels,
                                                  length = self.lookback,
                                                  batch_size = val_batch_size,
                                                  shuffle = False)
        test_generator = TimeseriesGenerator(test_data, test_labels,
                                            length = self.lookback,
                                            batch_size = test_batch_size,
                                            sampling_rate = sampling_rate,
                                            shuffle = False)    
        # Return the data generators
        return train_generator, validation_generator, test_generator
        
    def build_compile(self):
        
        assert type(self.epochs) == int
        assert type(self.lookback) == int
        assert type(self.layer_size) == int
        assert type(self.layer2_size) == int
        
        # Create LSTM
        model = Sequential()
        model.add(LSTM(self.layer_size,
                      activation = 'relu',
                      input_shape = (self.lookback, 66),
                      return_sequences = True))
        model.add(LSTM(self.layer2_size,
                      activation = 'relu',
                      input_shape = (self.lookback, 66)))
        # Ouput 1 value
        model.add(Dense(1))
        # Popular otimizer and loss function of mae
        model.compile(optimizer = 'ADAM', loss = 'mean_absolute_error')
        return model
    
    def fit_evaluate(self):
        # To save computational time after no improvement in loss 
        early_stopping = EarlyStopping(verbose = 1, patience = 3)
        # fit model
        self.model.fit(self.train_gen, validation_data = self.val_gen,
                       epochs = self.epochs,
                       verbose = 0,
                       callbacks = [early_stopping])
        
        # Since we want to minimize mae, we do 1/value
        evaluation = self.model.evaluate(self.val_gen)
        return 1 / evaluation
    
    def predictions(self):
        """A funciton to make predictions"""
        self.model.fit(self.train_gen, validation_data = self.val_gen,
                      epochs = self.epochs)
        stacked_lstm2_predictions = self.model.predict(self.test_gen, verbose = 0)
        # Returns target labels and predictions
        return stacked_lstm2_predictions, self.test_gen.targets

In [6]:
# Black box model
def stacked_lstm2_blackbox(epochs = 1, lookback = 5, 
                        layer_size = 64, layer2_size = 64):
    """A function to run and evaulate model
    """
    
    # Parametes to be optimized
    epochs = int(epochs)
    lookback = int(lookback)
    layer_size = int(layer_size)
    layer2_size = int(layer2_size)
    
    # Build model
    stacked_lstm2 = Stacked_LSTM2(epochs = epochs, lookback = lookback, 
                                  layer_size = layer_size,
                                  layer2_size = layer2_size)
    # Evaluate model
    stacked_lstm2_evaluation = stacked_lstm2.fit_evaluate()
    return stacked_lstm2_evaluation



In [7]:
# Bayes optimization for the parameter epochs
pbounds = {'epochs': (1,100),
          'lookback': (1,30),
          'layer_size': (1,256),
          'layer2_size': (1, 256)}
# Run bayesian optimization with specified bound
stacked_lstm2_optimizer = BayesianOptimization(f = stacked_lstm2_blackbox,
                                        pbounds = pbounds,
                                        verbose = 2,
                                        random_state = 1)

In [8]:
# Run optimization for a total of 50 iterations
stacked_lstm2_optimizer.maximize(init_points = 2,
                          n_iter = 8)
# Print model with optimized parameters
print(stacked_lstm2_optimizer.max)

|   iter    |  target   |  epochs   | layer2... | layer_... | lookback  |
-------------------------------------------------------------------------
Epoch 00011: early stopping
33/33 [==============================] - 0s 9ms/step - loss: 0.0047
|  1        |  214.6    |  42.29    |  184.7    |  1.029    |  9.768    |
Epoch 00012: early stopping
33/33 [==============================] - 0s 10ms/step - loss: 0.0122
|  2        |  81.73    |  15.53    |  24.55    |  48.5     |  11.02    |
Epoch 00007: early stopping
33/33 [==============================] - 0s 8ms/step - loss: 0.0093
|  3        |  107.1    |  42.31    |  184.5    |  7.393    |  8.237    |
Epoch 00004: early stopping
33/33 [==============================] - 0s 9ms/step - loss: 0.0045
|  4        |  223.7    |  46.98    |  185.7    |  1.741    |  9.205    |
Epoch 00009: early stopping
33/33 [==============================] - 0s 10ms/step - loss: 0.0090
|  5        |  110.6    |  46.44    |  184.3    |  2.238    |  12.71    |


In [11]:
# Build model with optimized parameters
stacked_lstm2_model = Stacked_LSTM2(epochs = 86, lookback = 1, 
                                     layer_size = 238,
                                     layer2_size = 98)
# Obtain predictions and actual
stacked_lstm2_pred, stacked_lstm2_actual = stacked_lstm2_model.predictions()

Train for 135 steps, validate for 34 steps
Epoch 1/86
135/135 [==============================] - 4s 31ms/step - loss: 0.0144 - val_loss: 0.0054
Epoch 2/86
135/135 [==============================] - 1s 9ms/step - loss: 0.0081 - val_loss: 0.0067
Epoch 3/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0076 - val_loss: 0.0046
Epoch 4/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0075 - val_loss: 0.0119
Epoch 5/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0076 - val_loss: 0.0060
Epoch 6/86
135/135 [==============================] - 1s 9ms/step - loss: 0.0072 - val_loss: 0.0063
Epoch 7/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0071 - val_loss: 0.0066
Epoch 8/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0070 - val_loss: 0.0053
Epoch 9/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0068 - val_loss: 0.0041
Epoch 10/86
135/135 [=============================

135/135 [==============================] - 1s 9ms/step - loss: 0.0031 - val_loss: 0.0059
Epoch 82/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0030 - val_loss: 0.0057
Epoch 83/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0027 - val_loss: 0.0059
Epoch 84/86
135/135 [==============================] - 1s 10ms/step - loss: 0.0028 - val_loss: 0.0057
Epoch 85/86
135/135 [==============================] - 1s 9ms/step - loss: 0.0028 - val_loss: 0.0055
Epoch 86/86
135/135 [==============================] - 1s 9ms/step - loss: 0.0028 - val_loss: 0.0054


In [13]:
# Plot predictions vs actual
plot_returns("Stacked LSTM",
            stacked_lstm2_actual,
             stacked_lstm2_pred,
            1)

In [14]:
# Calc mae on test data
calc_mae(stacked_lstm2_actual, stacked_lstm2_pred, 1)

0.003575831113735636